# TensorBoard

Tensorboard is a graph visualization software included with any standard TensorFlow installation. The computations you'll use TensorFlow for - like training a massive deep neural network - can be complex and confusing. To make it easier to understand, debug, and optimize TensorFlow programs, we've included a suite of visualization tools called TensorBoard. You can use TensorBoard to visualize your TensorFlow graph, plot quantitative metrics about the execution of your graph, and show additional data like images that pass through it.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

When a user perform certain operations in a TensorFlow program, these operations are exported to an event log file. TensorBoard is able to convert these event files to visualizations that can give insight into a model's graph and its runtime behavior. 

To visualize the program with TensorBoard, we need to write log files of the program. To write event files, we first need to create a writer for those logs, using this code:

writer = tf.summary.FileWriter([logdir], [graph])

[graph] is the graph of the program you are working on. You can either call it using tf.get_default_graph(), which returns the default graph of the program, or through sess.graph, which returns the graph the session is handling. The latter requires you to already have created a session. Either way, make sure to create a writer only after you’ve defined your graph, else the graph visualized on TensorBoard would be incomplete.

[logdir] is the folder where you want to store those log files. You can choose [logdir] to be something meaningful such as './graphs'. 

In [2]:
a = tf.constant(2)
b = tf.constant(3)
x = tf.add(a, b)
writer = tf.summary.FileWriter('./graphs', tf.get_default_graph())
with tf.Session() as sess:
    # writer = tf.summary.FileWriter('./graphs', sess.graph) # if you prefer creating your writer using session's graph
    print(sess.run(x))
writer.close()

5


## To launch tensorboard

In a terminal, type:
$ tensorboard --logdir="./graphs" --port 6006  

where logdir points to the directory where the FileWriter serialized its data. Once TensorBoard is running, navigate your web browser to:

http://localhost:6006/ 


## Naming elements of the graph
You will see the TensorBoard page. Note that if you don't give variables/constants/ops a name, then tensorflow will automatically generate one for you.  So, we can go back and name the constants and see what the graph looks like:

In [3]:
a = tf.constant(2, name="a")
b = tf.constant(2, name="b")
x = tf.add(a, b, name="add")
writer = tf.summary.FileWriter('./graphs', tf.get_default_graph())
with tf.Session() as sess:
    # writer = tf.summary.FileWriter('./graphs', sess.graph) # if you prefer creating your writer using session's graph
    print(sess.run(x))
writer.close()

4


## Warning

Notice that if you refresh tensorboard now, you will find two separate graphs.  You have to remember to reset the default graph if you want to maintain a single graph.

In [4]:
tf.reset_default_graph()

a = tf.constant(2, name="a")
b = tf.constant(2, name="b")
x = tf.add(a, b, name="add")
writer = tf.summary.FileWriter('./graphs', tf.get_default_graph())
with tf.Session() as sess:
    # writer = tf.summary.FileWriter('./graphs', sess.graph) # if you prefer creating your writer using session's graph
    print(sess.run(x))
writer.close()

4



### Name scoping and nodes

Typical TensorFlow graphs can have many thousands of nodes--far too many to see easily all at once, or even to lay out using standard graph tools. To simplify, variable names can be scoped and the visualization uses this information to define a hierarchy on the nodes in the graph. By default, only the top of this hierarchy is shown. Here is an example that defines three operations under the hidden name scope using tf.name_scope:

In [5]:
tf.reset_default_graph()

#writer = tf.summary.FileWriter('./graphs', tf.get_default_graph())
with tf.name_scope('hidden') as scope:
    a = tf.constant(5, name='alpha')
    W = tf.Variable(tf.random_uniform([1, 2], -1.0, 1.0), name='weights')
    b = tf.Variable(tf.zeros([1]), name='biases')



Grouping nodes by name scopes is critical to making a legible graph. If you're building a model, name scopes give you control over the resulting visualization. The better your name scopes, the better your visualization.

### TensorBoard Histogram Dashboard


The TensorBoard Histogram Dashboard displays how the distribution of some Tensor in your TensorFlow graph has changed over time. 

In [8]:
import tensorflow as tf
tf.reset_default_graph()

k = tf.placeholder(tf.float32)

# Make a normal distribution, with a shifting mean
mean_moving_normal = tf.random_normal(shape=[1000], mean=(5*k), stddev=1)

# Record that distribution into a histogram summary
tf.summary.histogram("normal/moving_mean", mean_moving_normal)

# Setup a session and summary writer
sess = tf.Session()
writer = tf.summary.FileWriter("./histogram_example")

summaries = tf.summary.merge_all()

# Setup a loop and write the summaries to disk
N = 400
for step in range(N):
    k_val = step/float(N)
    summ = sess.run(summaries, feed_dict={k: k_val})
    writer.add_summary(summ, global_step=step)

## To launch tensorboard

In a terminal, type:
$ tensorboard --logdir="./histogram_example" --port 6006  

where logdir points to the directory where the FileWriter serialized its data. Once TensorBoard is running, navigate your web browser to:

http://localhost:6006/ 

## Other notes

#### dependencies

TensorFlow graphs have two kinds of connections: data dependencies and control dependencies. Data dependencies show the flow of tensors between two ops and are shown as solid arrows, while control dependencies use dotted lines. 

#### chaning visuals

TensorBoard provides several ways to change the visual layout of the graph. This doesn't change the graph's computational semantics, but it can bring some clarity to the network's structure. By right clicking on a node or pressing buttons on the bottom of that node's info card, you can make the following changes to its layout.

### Runtime statistics

Often it is useful to collect runtime metadata for a run, such as total memory usage, total compute time, and tensor shapes for nodes. 

The TensorBoard Histogram Dashboard displays how the distribution of some Tensor in your TensorFlow graph has changed over time. It does this by showing many histograms visualizations of your tensor at different points in time.